In [52]:
season_data_frames = []

for year in range(2003, 2025):  # Include 2024
    print(f"Processing season {year}...")
    
    # Ensure the year suffix is two digits (e.g., '03' instead of '3')
    year_suffix = str(year)[-2:]  # Get the last two digits
    file_path = f"data/kenpom/summary{year_suffix}_pt.csv"
    
    # Read the efficiency stats for the season from the CSV file
    try:
        eff_stats = pd.read_csv(file_path)
        
        # Add a 'Season' column
        eff_stats['Season'] = year
        
        # Append the DataFrame to our list
        season_data_frames.append(eff_stats)
    except FileNotFoundError:
        print(f"File not found: {file_path}. Skipping...")
        continue

# Concatenate all season DataFrames into one
all_seasons_data = pd.concat(season_data_frames, ignore_index=True)

# Save the concatenated DataFrame to a CSV file
all_seasons_data.to_csv('kenpom_efficiency_stats_2003_to_2024.csv', index=False)

print("All seasons processed and saved.")


Processing season 2003...
Processing season 2004...
Processing season 2005...
Processing season 2006...
Processing season 2007...
Processing season 2008...
Processing season 2009...
Processing season 2010...
Processing season 2011...
Processing season 2012...
Processing season 2013...
Processing season 2014...
Processing season 2015...
Processing season 2016...
Processing season 2017...
Processing season 2018...
Processing season 2019...
Processing season 2020...
Processing season 2021...
Processing season 2022...
Processing season 2023...
Processing season 2024...
All seasons processed and saved.


In [22]:
def scrape_ap_poll(year):
    url = f"https://www.sports-reference.com/cbb/seasons/{year}-polls.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table = soup.find("table", {"id": "ap-polls"})
    # Extracting headers
    headers = ["School", "Conf"]
    headers.extend([f"Week_{i}" for i in range(1, 19)])  # Adjust according to the weeks you need
    
    # Initialize data storage
    data = []
    
    # Loop through each row in the table body
    for row in table.tbody.find_all("tr"):
        # Extract school name, ensuring we drill down to the <a> tag
        th = row.find("th", {"data-stat": "school"})
        if th and th.find("a"):
            school = th.find("a").text.strip()
        else:
            continue
        conf_cell = row.find("td", {"data-stat": "conf"})
        conf = conf_cell.text.strip() if conf_cell else 'Unknown Conf'
        
        row_data = [school, conf]
        
        # Extracting weekly ranks, with handling for missing data
        for week in range(1, 19):
            cell = row.find("td", {"data-stat": f"week{week}"})
            rank = cell.text.strip() if cell and cell.text.strip() else 100  # Use 'NR' (Not Ranked) if no data
            row_data.append(rank)
        
        data.append(row_data)
    
    # Creating DataFrame
    df = pd.DataFrame(data, columns=headers)
    df_reduced = df[['School', 'Conf', 'Week_1', 'Week_6', 'Week_12', 'Week_18']]

    return df_reduced


In [25]:
ap_poll = pd.DataFrame()

# Define the range of years you're interested in
years = range(2003, 2025)

for year in years:
    try:
        # Scrape data for the year
        df_year = scrape_ap_poll(year)
        # Add a 'Year' column to distinguish data from different years
        df_year['Season'] = year
        # Append the yearly data to the aggregated DataFrame
        ap_poll = pd.concat([ap_poll, df_year], ignore_index=True)
        
        print(f"Successfully scraped data for {year}")
    except Exception as e:
        print(f"Failed to scrape data for {year}: {e}")
    
    # Wait for 15 seconds before making the next request
    time.sleep(15)

ap_poll.to_csv('data/ap_poll.csv', index=False)

Successfully scraped data for 2003
Successfully scraped data for 2004
Successfully scraped data for 2005
Successfully scraped data for 2006
Successfully scraped data for 2007
Successfully scraped data for 2008
Successfully scraped data for 2009
Successfully scraped data for 2010
Successfully scraped data for 2011
Successfully scraped data for 2012
Successfully scraped data for 2013
Successfully scraped data for 2014
Successfully scraped data for 2015
Successfully scraped data for 2016
Successfully scraped data for 2017
Successfully scraped data for 2018
Successfully scraped data for 2019
Successfully scraped data for 2020
Successfully scraped data for 2021
Successfully scraped data for 2022
Successfully scraped data for 2023
Successfully scraped data for 2024


In [184]:
import pandas as pd
import numpy as np

In [185]:
RegSeason = pd.read_csv("data/MRegularSeasonDetailedResults.csv")
 
frames = [RegSeason]
games = pd.concat(frames)
games = games.sort_values(by=['Season', 'DayNum'])

# Assuming your games DataFrame is already defined and includes all necessary columns
games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113236,2024,132,1120,86,1196,67,N,0,31,61,...,13,18,25,13,26,9,11,8,6,17
113237,2024,132,1182,57,1433,51,N,0,17,57,...,24,12,14,9,25,9,16,10,9,16
113238,2024,132,1228,93,1458,87,N,0,30,57,...,20,20,23,13,17,17,7,7,1,20
113239,2024,132,1412,85,1396,69,N,0,31,63,...,21,14,17,11,26,17,14,6,6,18


In [186]:
WinnerStats = pd.DataFrame()
LoserStats = pd.DataFrame()

cols = ['Season', 'TeamID', 'PF', 'PA', 'Loc', 'NumOT',
 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO',
 'Stl', 'Blk', 'Fls', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR',
 'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppFls']

WinnerStats[cols] = games[['Season', 'WTeamID', 'WScore', 'LScore', 'WLoc', 'NumOT',
 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR','WAst', 'WTO',
 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR',
 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]

WinnerStats['Wins'] = 1
WinnerStats['Loses'] = 0

LoserStats[cols] = games[['Season', 'LTeamID', 'LScore', 'WScore', 'WLoc', 'NumOT',
 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR','LAst', 'LTO',
 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR',
 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

LoserStats['Loc'] = LoserStats['Loc'].replace({'H': 'A', 'A': 'H'})

LoserStats['Wins'] = 0
LoserStats['Loses'] = 1

WinLose = pd.concat([WinnerStats, LoserStats])

WinLose.columns

Index(['Season', 'TeamID', 'PF', 'PA', 'Loc', 'NumOT', 'FGM', 'FGA', 'FGM3',
       'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'Fls',
       'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR',
       'OppDR', 'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppFls', 'Wins',
       'Loses'],
      dtype='object')

In [187]:
TeamStats = WinLose.groupby(['Season', 'TeamID']).sum()

TeamStats['Games Played'] = TeamStats['Wins'] + TeamStats['Loses']
TeamStats['Minutes Played'] = TeamStats['Games Played'] * 40 + TeamStats['NumOT'] * 5
TeamStats['Possesions'] = (TeamStats['FGA'] - TeamStats['OR']) + TeamStats['TO'] + (.44 * TeamStats['FTA'])
TeamStats['OppPossesions'] = (TeamStats['OppFGA'] - TeamStats['OppOR']) + TeamStats['OppTO'] + (.44 * TeamStats['OppFTA'])
TeamStats['Tempo'] = (TeamStats['Possesions'] / TeamStats['Minutes Played']) * 40
TeamStats['Win %'] = TeamStats['Wins'] / TeamStats['Games Played']
TeamStats['PPPos'] = TeamStats['PF'] / TeamStats['Possesions']
TeamStats['PAPPos'] = TeamStats['PA'] / TeamStats['OppPossesions']
TeamStats['PPPos Margin'] = TeamStats['PPPos'] - TeamStats['PAPPos']
TeamStats['FG %'] = TeamStats['FGM'] / TeamStats['FGA']
TeamStats['OppFG%'] = TeamStats['OppFGM'] / TeamStats['OppFGA']
TeamStats['3PT FG %'] = TeamStats['FGM3'] / TeamStats['FGA3']
TeamStats['3PTPPos'] = TeamStats['FGM3'] / TeamStats['Possesions']
TeamStats['FT %'] = TeamStats['FTM'] / TeamStats['FTA']
TeamStats['FTPPos'] = TeamStats['FTM'] / TeamStats['Possesions']
TeamStats['OppFTPPos'] = TeamStats['OppFTM'] / TeamStats['OppPossesions']
TeamStats['ORPPos'] = TeamStats['OR'] / TeamStats['Possesions']
TeamStats['Opp ORPPos'] = TeamStats['OppOR'] / TeamStats['OppPossesions']
TeamStats['DRPG'] = TeamStats['DR'] / TeamStats['Games Played']
TeamStats['REBPG'] = (TeamStats['OR'] + TeamStats['DR']) / TeamStats['Games Played']
TeamStats['REB Margin'] = (TeamStats['REBPG']) - ((TeamStats['OppOR'] + TeamStats['OppDR']) / TeamStats['Games Played'])
TeamStats['True Shooting %'] = (.5 * TeamStats['PF']) / (TeamStats['FGA'] + .475 * TeamStats['FTA'])
TeamStats['Effective FG%'] = (TeamStats['FGM'] + .5 * TeamStats['FGM3']) / TeamStats['FGA']
TeamStats['TOV %'] = TeamStats['TO'] / TeamStats['Possesions']
TeamStats['TOV Forced %'] = (TeamStats['Blk'] + TeamStats['Stl']) / TeamStats['OppPossesions']
TeamStats['Foul Margin'] = (TeamStats['Fls'] - TeamStats['OppFls']) / TeamStats['Games Played']
TeamStats['OppEFG'] = (TeamStats['OppFGM'] + 0.5 * TeamStats['OppFGM3']) / TeamStats['OppFGA']

/var/folders/bm/x9qxxjrx0fx_4fw8sspqvjgh0000gn/T/ipykernel_7315/3215041156.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  TeamStats = WinLose.groupby(['Season', 'TeamID']).sum()


In [188]:
TeamStats

PF    PA  NumOT  FGM   FGA  FGM3  FGA3  FTM  FTA   OR  ...  \
Season TeamID                                                           ...   
2003   1102    1603  1596      0  536  1114   219   583  312  479  117  ...   
       1103    2127  2110      8  733  1508   147   434  514  698  264  ...   
       1104    1940  1820      1  673  1601   178   556  416  586  380  ...   
       1105    1866  1993      4  634  1602   197   540  401  568  351  ...   
       1106    1781  1785      1  656  1548   171   494  298  461  344  ...   
...             ...   ...    ...  ...   ...   ...   ...  ...  ...  ...  ...   
2024   1474    2397  2560      0  836  1938   261   791  464  653  268  ...   
       1475    1941  2174      1  649  1630   187   591  456  613  246  ...   
       1476    1896  2316      1  688  1738   246   816  274  393  206  ...   
       1477    2018  2298      9  699  1782   241   820  379  548  225  ...   
       1478    2029  2073      1  713  1697   275   806  328  429  206  ...   

               Opp ORPPos       DRPG      REBPG  REB Margin  True Shooting %  \
Season TeamID                                                                  
2003   1102      0.177072  16.821429  21.000000   -8.750000         0.597454   
       1103      0.171856  19.925926  29.703704   -4.370370         0.578131   
       1104      0.165025  23.928571  37.500000    3.964286         0.516136   
       1105      0.175064  23.115385  36.615385   -2.961538         0.498451   
       1106      0.169464  23.857143  36.142857    2.464286         0.503969   
...                   ...        ...        ...         ...              ...   
2024   1474      0.123217  23.580645  32.225806   -2.741935         0.533099   
       1475      0.120599  24.206897  32.689655   -0.206897         0.505160   
       1476      0.139601  23.066667  29.933333   -6.666667         0.492551   
       1477      0.134477  20.333333  27.833333   -8.900000         0.494051   
       1478      0.136014  23.620690  30.724138   -4.310345         0.533730   

               Effective FG%     TOV %  TOV Forced %  Foul Margin    OppEFG  
Season TeamID                                                                
2003   1102         0.579443  0.209457      0.142842     0.392857  0.510522  
       1103         0.534814  0.180221      0.136956    -2.592593  0.545809  
       1104         0.475953  0.200990      0.157450    -1.214286  0.476190  
       1105         0.457241  0.244219      0.151076     1.153846  0.511089  
       1106         0.479005  0.253206      0.172137     2.035714  0.451505  
...                      ...       ...           ...          ...       ...  
2024   1474         0.498710  0.150587      0.126713    -0.677419  0.541224  
       1475         0.455521  0.183638      0.126999     2.827586  0.495034  
       1476         0.466628  0.185776      0.123401     1.400000  0.527183  
       1477         0.459877  0.155066      0.169517     2.366667  0.524306  
       1478         0.501179  0.156645      0.148562     1.517241  0.506122  

[7617 rows x 58 columns]

In [189]:
def calculate_last_6_games_averages(games):
    stats_of_interest = ['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'Score', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
    
    uniform_games = []

    # Process winning and losing stats
    for index, row in games.iterrows():
        # Winning stats
        win_stats = {stat: row[f'W{stat}'] for stat in stats_of_interest}
        win_stats['TeamID'] = row['WTeamID']
        win_stats['Season'] = row['Season']
        win_stats['Win'] = 1
        win_stats['Possessions'] = (row['WFGA'] - row['WOR']) + row['WTO'] + (.44 * row['WFTA'])
        uniform_games.append(win_stats)
        
        # Losing stats
        lose_stats = {stat: row[f'L{stat}'] for stat in stats_of_interest}
        lose_stats['TeamID'] = row['LTeamID']
        lose_stats['Season'] = row['Season']
        lose_stats['Win'] = 0
        lose_stats['Possessions'] = (row['LFGA'] - row['LOR']) + row['LTO'] + (.44 * row['LFTA'])
        uniform_games.append(lose_stats)

    # Create a DataFrame from the uniform games list
    uniform_df = pd.DataFrame(uniform_games)
    
    # Calculate per-possession stats for each stat of interest
    for stat in stats_of_interest:
        uniform_df[f'{stat}_per_poss'] = uniform_df[stat] / uniform_df['Possessions']

    # Group by TeamID and Season, then calculate the mean for the last 6 games
    grouped = uniform_df.groupby(['TeamID', 'Season'])
    last_6_games_avg = grouped.tail(6).groupby(['TeamID', 'Season']).mean().reset_index()

    # Prepare last_6_games_avg for merging by renaming columns
    rename_columns = {stat: f'{stat}_last_6' for stat in last_6_games_avg.columns if stat not in ['TeamID', 'Season']}
    last_6_games_avg.rename(columns=rename_columns, inplace=True)
    
    return last_6_games_avg

In [190]:
# Calculate the last 6 games' per-possession averages
last_6_games_stats = calculate_last_6_games_averages(games)
last_6_games_stats

,TeamID,Season,FGM_last_6,FGA_last_6,FGM3_last_6,FGA3_last_6,FTM_last_6,FTA_last_6,Score_last_6,OR_last_6,...,FTM_per_poss_last_6,FTA_per_poss_last_6,Score_per_poss_last_6,OR_per_poss_last_6,DR_per_poss_last_6,Ast_per_poss_last_6,TO_per_poss_last_6,Stl_per_poss_last_6,Blk_per_poss_last_6,PF_per_poss_last_6
0,1101,2014,22.833333,45.666667,7.833333,17.000000,18.333333,23.333333,71.833333,5.833333,...,0.279715,0.357034,1.090156,0.088999,0.313002,0.186661,0.236770,0.089496,0.019671,0.310486
1,1101,2015,19.500000,50.833333,4.833333,15.500000,8.666667,11.000000,52.500000,6.333333,...,0.141513,0.180149,0.856844,0.104004,0.319132,0.175323,0.195207,0.100953,0.005468,0.337961
2,1101,2016,25.833333,53.333333,8.000000,20.000000,14.833333,21.500000,74.500000,7.333333,...,0.212906,0.306583,1.075086,0.105216,0.354600,0.201832,0.199368,0.093382,0.033411,0.345009
3,1101,2017,26.166667,56.166667,7.833333,20.166667,12.000000,17.333333,72.166667,8.833333,...,0.171279,0.246235,1.041326,0.126555,0.341185,0.228400,0.204807,0.100521,0.033684,0.299247
4,1101,2018,25.000000,58.333333,5.500000,19.166667,10.833333,16.500000,66.333333,12.666667,...,0.160744,0.243166,0.977643,0.187634,0.344665,0.174533,0.219990,0.110102,0.039119,0.275756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,1476,2023,22.333333,53.166667,5.833333,18.333333,12.666667,16.000000,63.166667,7.000000,...,0.194659,0.247261,0.961925,0.102855,0.359837,0.179958,0.186462,0.100820,0.053142,0.209018
7613,1476,2024,24.000000,58.000000,7.500000,27.333333,8.333333,12.666667,63.833333,7.333333,...,0.128697,0.195244,0.959487,0.112049,0.372186,0.202153,0.154039,0.047732,0.041006,0.214500
7614,1477,2023,28.500000,56.666667,8.833333,23.000000,13.000000,18.666667,78.833333,6.500000,...,0.184515,0.264497,1.116034,0.092558,0.280352,0.202897,0.175627,0.082901,0.037026,0.257683
7615,1477,2024,23.833333,59.166667,5.833333,23.333333,17.166667,25.000000,70.666667,8.500000,...,0.240291,0.347078,0.994641,0.116309,0.300993,0.131397,0.137762,0.100250,0.053501,0.228334


In [191]:
import requests
from bs4 import BeautifulSoup

In [192]:
TeamStats.reset_index(inplace=True)

TeamStats = TeamStats.merge(last_6_games_stats, on=['TeamID', 'Season'], how='left')

TeamStats.rename(columns=lambda x: x if '_last_6' in x and x == 'Win_last_6' else (x + '_last_6' if '_last_6' in x else x), inplace=True)

In [193]:
columns_to_remove = ['FGM_last_6_last_6', 'FGA_last_6_last_6', 'FGM3_last_6_last_6',
       'FGA3_last_6_last_6', 'FTM_last_6_last_6', 'FTA_last_6_last_6',
       'Score_last_6_last_6', 'OR_last_6_last_6', 'DR_last_6_last_6',
       'Ast_last_6_last_6', 'TO_last_6_last_6', 'Stl_last_6_last_6',
       'Blk_last_6_last_6', 'PF_last_6_last_6']
TeamStats.drop(columns=columns_to_remove, inplace=True)

In [194]:
TeamStats.columns

Index(['Season', 'TeamID', 'PF', 'PA', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'Fls', 'OppFGM',
       'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR',
       'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppFls', 'Wins', 'Loses',
       'Games Played', 'Minutes Played', 'Possesions', 'OppPossesions',
       'Tempo', 'Win %', 'PPPos', 'PAPPos', 'PPPos Margin', 'FG %', 'OppFG%',
       '3PT FG %', '3PTPPos', 'FT %', 'FTPPos', 'OppFTPPos', 'ORPPos',
       'Opp ORPPos', 'DRPG', 'REBPG', 'REB Margin', 'True Shooting %',
       'Effective FG%', 'TOV %', 'TOV Forced %', 'Foul Margin', 'OppEFG',
       'Win_last_6', 'Possessions_last_6_last_6', 'FGM_per_poss_last_6_last_6',
       'FGA_per_poss_last_6_last_6', 'FGM3_per_poss_last_6_last_6',
       'FGA3_per_poss_last_6_last_6', 'FTM_per_poss_last_6_last_6',
       'FTA_per_poss_last_6_last_6', 'Score_per_poss_last_6_last_6',
       'OR_per_poss_last_6_last_6', 'DR_per_

In [195]:
from kenpompy.utils import login
import kenpompy.summary as kp
import time
# Returns an authenticated browser that can then be used to scrape pages that require authorization.
browser = login('rami.alkadri@gmail.com', 'Rambonius254426!')

In [196]:
ap_poll = pd.read_csv("data/ap_poll.csv")
ap_poll

unique_elements = teams_df['TeamName'].unique()

print(unique_elements)


['Abilene Chr' 'Air Force' 'Akron' 'Alabama' 'Alabama A&M' 'Alabama St'
 'SUNY Albany' 'Alcorn St' 'Alliant Intl' 'American Univ' 'Appalachian St'
 'Arizona' 'Arizona St' 'Ark Little Rock' 'Ark Pine Bluff' 'Arkansas'
 'Arkansas St' 'Armstrong St' 'Army' 'Auburn' 'Augusta' 'Austin Peay'
 'Ball St' 'Baylor' 'Belmont' 'Bethune-Cookman' 'Binghamton'
 'Birmingham So' 'Boise St' 'Boston College' 'Boston Univ' 'Bowling Green'
 'Bradley' 'Brooklyn' 'Brown' 'Bryant' 'Bucknell' 'Buffalo' 'Butler' 'BYU'
 'C Michigan' 'Cal Poly' 'California' 'Campbell' 'Canisius'
 'Cent Arkansas' 'Centenary' 'Central Conn' 'Charleston So' 'Charlotte'
 'Chattanooga' 'Chicago St' 'Cincinnati' 'Citadel' 'Clemson'
 'Cleveland St' 'Coastal Car' 'Col Charleston' 'Colgate' 'Colorado'
 'Colorado St' 'Columbia' 'Connecticut' 'Coppin St' 'Cornell' 'Creighton'
 'CS Bakersfield' 'CS Fullerton' 'CS Northridge' 'CS Sacramento'
 'Dartmouth' 'Davidson' 'Dayton' 'Delaware' 'Delaware St' 'Denver'
 'DePaul' 'Detroit' 'Drake' 'Drexel

In [197]:
name_corrections = {
'College of Charleston' : 'Col Charleston',
'Florida Atlantic' : 'FL Atlantic',
'George Washington' : 'G Washington',
'Loyola (IL)' : 'Loyola-Chicago',
'Middle Tennessee' : 'MTSU',
'NC State' : 'NC St',
'Ole Miss' : 'Mississippi',
'Pitt' : 'Pittsburgh',
'Saint Louis' : 'St Louis',
"Saint Mary's" : "St Mary's CA",
"St. John's (NY)" : "St John's",
'UConn' : "Connecticut",
'UMass' : "Massachusetts",
'UNC' : "North Carolina",
'Miami (FL)' : 'Miami FL',
'Kent State': 'Kent',
'Southern Illinois': 'S Illinois',
'St. Bonaventure' : 'St Bonaventure',
"St. Joseph's" : "St Joseph's PA", 
'Western Kentucky' : "WKU"
}

def apply_name_corrections_sr(df, corrections):
    for wrong_name, correct_name in corrections.items():
        df['School'] = df['School'].replace(wrong_name, correct_name)
    return df

# Apply corrections to kenpom
ap_poll = apply_name_corrections_sr(ap_poll, name_corrections)

In [198]:
teams_df = pd.read_csv('data/MTeams.csv')
# Initialize a set to collect unmatched team names
unmatched_teams = set()

# Iterate over kenpom DataFrame to match team names with teams_df
for index, row in ap_poll.iterrows():
    # Preprocess the team name from kenpom for comparison
    # Remove '.', strip whitespace, and convert to lowercase
    preprocessed_team_name = row['School'].replace('State', 'St').strip().lower()
    
    # Check if the preprocessed team name exists in teams_df (also preprocessed)
    # Note: We assume TeamName in teams_df is the column to match with
    if not teams_df['TeamName'].str.replace('.', '').str.strip().str.lower().str.contains('^' + preprocessed_team_name + '$').any():
        # If no match is found, add the original team name to the unmatched_teams set
        unmatched_teams.add(row['School'])

# Print out unmatched team names
print("Unmatched team names:")
for team in sorted(unmatched_teams):
    print(f"\'{team}\'")
    
print(len(unmatched_teams))

Unmatched team names:

/var/folders/bm/x9qxxjrx0fx_4fw8sspqvjgh0000gn/T/ipykernel_7315/1520749835.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if not teams_df['TeamName'].str.replace('.', '').str.strip().str.lower().str.contains('^' + preprocessed_team_name + '$').any():



'NC St'
1


In [199]:
def preprocess_team_name(name):
    return name.replace('State', 'St').strip()

# Apply preprocessing to team names in all_seasons_data
ap_poll['School'] = ap_poll['School'].apply(preprocess_team_name)

# Apply preprocessing to team names in teams_df
teams_df['TeamName'] = teams_df['TeamName'].apply(preprocess_team_name)

# Now proceed with the matching logic
unmatched_teams = set()

for index, row in ap_poll.iterrows():
    team_name = row['School']
    matched_team = teams_df[teams_df['TeamName'] == team_name]
    
    if not matched_team.empty:
        # If a match is found, assign the TeamID
        ap_poll.at[index, 'TeamID'] = matched_team.iloc[0]['TeamID']
    else:
        # If no match is found, print the comparison and add to the unmatched_teams set
        print(f"Unmatched team in all_seasons_data: '{team_name}'")
        unmatched_teams.add(team_name)

In [200]:
# Convert Team_ID in all_seasons_data to pandas' nullable integer type
ap_poll['TeamID'] = ap_poll['TeamID'].astype('Int64')

# # # Ensure TeamID in TeamStats is also the correct type
TeamStats['TeamID'] = TeamStats['TeamID'].astype('Int64')

# # # Merge all_seasons_data into TeamStats
# # # Assuming 'Season' is a column in both and 'Team_ID' in all_seasons_data matches 'TeamID' in TeamStats
TeamStats = pd.merge(TeamStats, ap_poll, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')

# # # If you want to set ['Season', 'TeamID'] back as index after merge
TeamStats.groupby(['TeamID', 'Season'])
print(TeamStats.columns)

Index(['Season', 'TeamID', 'PF', 'PA', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'Fls', 'OppFGM',
       'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR',
       'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppFls', 'Wins', 'Loses',
       'Games Played', 'Minutes Played', 'Possesions', 'OppPossesions',
       'Tempo', 'Win %', 'PPPos', 'PAPPos', 'PPPos Margin', 'FG %', 'OppFG%',
       '3PT FG %', '3PTPPos', 'FT %', 'FTPPos', 'OppFTPPos', 'ORPPos',
       'Opp ORPPos', 'DRPG', 'REBPG', 'REB Margin', 'True Shooting %',
       'Effective FG%', 'TOV %', 'TOV Forced %', 'Foul Margin', 'OppEFG',
       'Win_last_6', 'Possessions_last_6_last_6', 'FGM_per_poss_last_6_last_6',
       'FGA_per_poss_last_6_last_6', 'FGM3_per_poss_last_6_last_6',
       'FGA3_per_poss_last_6_last_6', 'FTM_per_poss_last_6_last_6',
       'FTA_per_poss_last_6_last_6', 'Score_per_poss_last_6_last_6',
       'OR_per_poss_last_6_last_6', 'DR_per_

In [201]:
kenpom = pd.read_csv('data/kenpom_efficiency_stats_2003_to_2024.csv')


print("Original kenpom entries for N.C. State:")
print(teams_df[teams_df['TeamName'].isin(['NC St'])])


Original kenpom entries for N.C. State:
     TeamID TeamName  FirstD1Season  LastD1Season
200    1301    NC St           1985          2024


In [202]:
name_corrections = {
    'Abilene Christian' : 'Abilene Chr',
    'Albany' : 'SUNY Albany',
    'American' : 'American Univ',
    'Arkansas Little Rock' : 'Ark Little Rock',
    'Arkansas Pine Bluff' : 'Ark Pine Bluff', 
    'Bethune Cookman' : 'Bethune-Cookman',
    'Birmingham Southern': 'Birmingham So',
    'Boston University' : 'Boston Univ',
    'Cal St. Bakersfield' : 'CS Bakersfield',
    'Cal St. Fullerton' : 'CS Fullerton',
    'Cal St. Northridge': 'CS Northridge',
    'Central Arkansas' : 'Cent Arkansas',
    'Central Connecticut': 'Central Conn',
    'Central Michigan' : 'C Michigan',
    'Charleston' : 'Col Charleston',
    'Charleston Southern' : 'Charleston So',
    'Coastal Carolina' : 'Coastal Car',
    'College of Charleston' : 'Col Charleston',
    'Detroit Mercy' : 'Detroit',
    'Dixie St.' : 'Utah Tech',
    'East Tennessee St.' : 'ETSU',
    'Eastern Illinois' : 'E Illinois',
    'Eastern Kentucky' : 'E Kentucky',
    'Eastern Michigan' : 'E Michigan',
    'Eastern Washington' : 'E Washington',
    'FIU' : 'Florida Intl',
    'Fairleigh Dickinson' : 'F Dickinson',
    'Florida Atlantic' : 'FL Atlantic',
    'Florida Gulf Coast' : 'FL Gulf Coast',
    'Fort Wayne' : 'PFW',
    'George Washington' : 'G Washington',
    'Georgia Southern' : 'Ga Southern',
    'Grambling St.' : 'Grambling',
    'Green Bay' : 'WI Green Bay',
    'Houston Baptist' : 'Houston Chr',
    'Houston Christian' : 'Houston Chr',
    'IPFW' : 'PFW', 
    'Illinois Chicago' : 'IL Chicago',
    'Kennesaw St.' : 'Kennesaw',
    'Kent St.' : 'Kent',
    'LIU' : 'LIU Brooklyn',
    'Little Rock' : 'Ark Little Rock',
    'Louisiana Lafayette' : 'Lafayette',
    'Louisiana Monroe' : 'ULM',
    'Loyola Chicago' : 'Loyola-Chicago',
    'Loyola Marymount' : 'Loy Marymount',
    'Maryland Eastern Shore' : 'MD E Shore',
    'Middle Tennessee' : 'MTSU',
    'Middle Tennessee St.' : 'MTSU',
    'Milwaukee' : 'WI Milwaukee',
    'Mississippi Valley St.' : 'MS Valley St',
    'Monmouth' : 'Monmouth NJ',
    "Mount St. Mary's" : "Mt St Mary's",
    'Nebraska Omaha' : 'NE Omaha',
    'North Carolina A&T' : 'NC A&T',
    'North Carolina Central' : 'NC Central',
    'North Dakota St.' : 'N Dakota St',
    'Northern Colorado' : 'N Colorado',
    'Northern Illinois' : 'N Illinois',
    'Northern Kentucky' : 'N Kentucky',
    'Northwestern St.' : 'Northwestern LA',
    'Prairie View A&M' : 'Prairie View',
    'Purdue Fort Wayne' : 'PFW',
    'Queens' : 'Queens NC',
    'SIU Edwardsville': 'SIUE',
    'Sacramento St.' : 'CS Sacramento',
    "Saint Joseph's" : "St Joseph's PA",
    'Saint Louis' : 'St Louis',
    "Saint Mary's" : "St Mary's CA",
    "Saint Peter's" : "St Peter's",
    'South Carolina St.' : 'S Carolina St',
    'South Dakota St.' : 'S Dakota St',
    'Southeast Missouri St.' : 'SE Missouri St',
    'Southeastern Louisiana' : 'SE Louisiana',
    'Southern' : 'Southern Univ',
    'Southern Illinois' : 'S Illinois',
    'Southwest Missouri St.' : 'Missouri St',
    'Southwest Texas St.' : 'Texas St',
    'St. Thomas' : 'St Thomas MN',
    'Stephen F. Austin' : 'SF Austin',
    'Tennessee Martin' : 'TN Martin',
    'Texas A&M Commerce' : 'TX A&M Commerce',
    'Texas A&M Corpus Chris' : 'TAM C. Christi',
    'Texas Pan American' : 'UTRGV',
    'Texas Southern' : 'TX Southern',
    'The Citadel' : 'Citadel',
    'Troy St.' : 'Troy', 
    'UMKC' : 'Missouri KC',
    'UMass Lowell' : 'MA Lowell',
    'USC Upstate' : 'SC Upstate',
    'UT Rio Grande Valley' : 'UTRGV',
    'UTSA' : 'UT San Antonio',
    'Utah Valley St.' : 'Utah Valley',
    'Western Carolina' : 'W Carolina',
    'Western Illinois' : 'W Illinois',
    'Western Kentucky' : 'WKU',
    'Western Michigan' : 'W Michigan',
    'Winston Salem St.' : 'W Salem St',
    'N.C. State' : 'NC State',
    'North Carolina St.' : 'NC State',
    'Saint Francis': 'St Francis NY'
}

def apply_name_corrections(df, corrections):
    for wrong_name, correct_name in corrections.items():
        df['TeamName'] = df['TeamName'].replace(wrong_name, correct_name)
    return df

# Apply corrections to kenpom
kenpom = apply_name_corrections(kenpom, name_corrections)



In [203]:
# Initialize a set to collect unmatched team names
unmatched_teams = set()

# Iterate over kenpom DataFrame to match team names with teams_df
for index, row in kenpom.iterrows():
    # Preprocess the team name from kenpom for comparison
    # Remove '.', strip whitespace, and convert to lowercase
    preprocessed_team_name = row['TeamName'].replace('.', '').strip().lower()
    
    # Check if the preprocessed team name exists in teams_df (also preprocessed)
    # Note: We assume TeamName in teams_df is the column to match with
    if not teams_df['TeamName'].str.replace('.', '').str.strip().str.lower().str.contains('^' + preprocessed_team_name + '$').any():
        # If no match is found, add the original team name to the unmatched_teams set
        unmatched_teams.add(row['TeamName'])

# Print out unmatched team names
print("Unmatched team names:")
for team in sorted(unmatched_teams):
    print(f"\'{team}\'")
    
print(len(unmatched_teams))

/var/folders/bm/x9qxxjrx0fx_4fw8sspqvjgh0000gn/T/ipykernel_7315/3880465414.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if not teams_df['TeamName'].str.replace('.', '').str.strip().str.lower().str.contains('^' + preprocessed_team_name + '$').any():


Unmatched team names:
'NC State'
1


In [204]:
def preprocess_team_name(name):
    # Preprocess team names by removing dots and stripping whitespace
    return name.replace('.', '').strip()

# Apply preprocessing to team names in kenpom
kenpom['TeamName'] = kenpom['TeamName'].apply(preprocess_team_name)

# Apply preprocessing to team names in teams_df
teams_df['TeamName'] = teams_df['TeamName'].apply(preprocess_team_name)

# Handling the hardcoded case for 'NC State'
hardcoded_teams = {
    'NC State': 1301,
    'North Carolina St': 1301,
}

# Now proceed with the matching logic
unmatched_teams = set()

for index, row in kenpom.iterrows():
    team_name = row['TeamName']
    
    # Check if the team name matches one of the hardcoded cases
    if team_name in hardcoded_teams:
        kenpom.at[index, 'TeamID'] = hardcoded_teams[team_name]
    else:
        matched_team = teams_df[teams_df['TeamName'] == team_name]
        if not matched_team.empty:
            # If a match is found in teams_df, assign the TeamID
            kenpom.at[index, 'TeamID'] = matched_team.iloc[0]['TeamID']
        else:
            # If no match is found, print the unmatched team and add to the set
            print(f"Unmatched team in kenpom: '{team_name}'")
            unmatched_teams.add(team_name)

print("Processing complete.")


Processing complete.


In [205]:
# Convert Team_ID in all_seasons_data to pandas' nullable integer type
kenpom['TeamID'] = kenpom['TeamID'].astype('Int64')

# # # Ensure TeamID in TeamStats is also the correct type
TeamStats['TeamID'] = TeamStats['TeamID'].astype('Int64')

# # # Merge all_seasons_data into TeamStats
# # # Assuming 'Season' is a column in both and 'Team_ID' in all_seasons_data matches 'TeamID' in TeamStats
TeamStats = pd.merge(TeamStats, kenpom, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')

# # # If you want to set ['Season', 'TeamID'] back as index after merge
TeamStats.groupby(['Season', 'TeamID'])

In [206]:
TeamStats.dropna(subset=['TeamName'], inplace=True)
columns_to_remove = ['School', 'Conf', 'seed']
TeamStats.fillna(100, inplace=True)
#TeamStats.drop(columns=columns_to_remove, inplace=True)



In [207]:
# Filter the dataset for years 2003 to 2022
team_stats_2003_2022 = TeamStats[TeamStats['Season'] <= 2023]
# Save to CSV
team_stats_2003_2022.to_csv("data/teamData.csv", index=False)

# Filter the dataset for the year 2023
team_stats_2023 = TeamStats[TeamStats['Season'] == 2024]
# Save to CSV
team_stats_2023.to_csv("data/2024_teamData.csv", index=False)

In [210]:
TeamStats.columns

Index(['Season', 'TeamID', 'PF', 'PA', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'Fls', 'OppFGM',
       'OppFGA', 'OppFGM3', 'OppFGA3', 'OppFTM', 'OppFTA', 'OppOR', 'OppDR',
       'OppAst', 'OppTO', 'OppStl', 'OppBlk', 'OppFls', 'Wins', 'Loses',
       'Games Played', 'Minutes Played', 'Possesions', 'OppPossesions',
       'Tempo_x', 'Win %', 'PPPos', 'PAPPos', 'PPPos Margin', 'FG %', 'OppFG%',
       '3PT FG %', '3PTPPos', 'FT %', 'FTPPos', 'OppFTPPos', 'ORPPos',
       'Opp ORPPos', 'DRPG', 'REBPG', 'REB Margin', 'True Shooting %',
       'Effective FG%', 'TOV %', 'TOV Forced %', 'Foul Margin', 'OppEFG',
       'Win_last_6', 'Possessions_last_6_last_6', 'FGM_per_poss_last_6_last_6',
       'FGA_per_poss_last_6_last_6', 'FGM3_per_poss_last_6_last_6',
       'FGA3_per_poss_last_6_last_6', 'FTM_per_poss_last_6_last_6',
       'FTA_per_poss_last_6_last_6', 'Score_per_poss_last_6_last_6',
       'OR_per_poss_last_6_last_6', 'DR_pe

In [212]:
TeamStats['Score_per_poss_last_6_last_6']

0       0.993541
1       1.165932
2       1.087661
3       0.998121
4       0.902536
          ...   
7628    1.079779
7629    1.008304
7630    0.959487
7631    0.994641
7632    0.992263
Name: Score_per_poss_last_6_last_6, Length: 7612, dtype: float64